In [175]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# print(os.listdir("../input/test"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'train_labels.csv', 'sample_submission.csv']


In [176]:
TRAIN_DIR = "../input/train"
TEST_DIR = "../input/test"
IMG_SIZE = 64
training_data = None
testing_data = None

In [177]:
df = pd.read_csv("../input/train_labels.csv")
print(df.head())
print(df.tail())

   name  invasive
0     1         0
1     2         0
2     3         1
3     4         0
4     5         1
      name  invasive
2290  2291         1
2291  2292         1
2292  2293         1
2293  2294         1
2294  2295         1


# Preparing the training data

In [178]:
import cv2
import os
from random import shuffle
from tqdm import tqdm_notebook

def create_train_data():
    training_data = []
    for img in tqdm_notebook(os.listdir(TRAIN_DIR)):
        label = df.invasive[int(img.split('.')[0])-1]
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        # Normalizing the data and appending to training_data list
        training_data.append([np.array(img/255),label])
    shuffle(training_data)
    return training_data

def create_test_data():
    testing_data = []
    for img in tqdm_notebook(os.listdir(TEST_DIR)):
        name = img.split('.')[0]
        path = os.path.join(TEST_DIR, img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([name, np.array(img)/255])
    return testing_data
    
if training_data is None:
    training_data = create_train_data()
    
if testing_data is None:
    testing_data = create_test_data()

A Jupyter Widget

A Jupyter Widget

# Train a CNN

## Importing required Libraries

In [205]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

In [218]:
# Initialising the CNN
batch_size = 512
keep_prob = 0.75
classifier = Sequential()

In [219]:
# Step 1 - Convolution
classifier.add(Convolution2D(64, 3, 3, input_shape = (IMG_SIZE, IMG_SIZE , 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  


In [220]:
# Adding a second convolutional layer
classifier.add(Convolution2D(128, 3, 3, activation = 'relu'))
classifier.add(Dropout(keep_prob))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(keep_prob))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  


In [221]:
# Step 3 - Flattening
classifier.add(Flatten())

In [222]:
# Step 4 - Full connection
classifier.add(Dense(activation='relu', units=500))
classifier.add(Dropout(keep_prob))
classifier.add(Dense(activation='relu', units=500))
classifier.add(Dropout(keep_prob))
classifier.add(Dense(activation='sigmoid', units=1))

In [223]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Creating train and validation split

In [224]:
split_size = 1800
train = training_data[:split_size]
validation = training_data[split_size:]

In [225]:
# train_x = np.array(train).transpose()[0]
# train_y = np.array(train).transpose()[1]

train_x = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
# train_y = [i[1] for i in train]
train_y = np.array(train).transpose()[1]

validation_x = np.array([i[0] for i in validation]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
validation_y = np.array(validation).transpose()[1]

testing_x = np.array([i[1] for i in testing_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
testing_name = np.array([i[0] for i in testing_data])
# testing_x = np.array(testing_data)
# testing_x = np.array(i for i in testing_data).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [226]:
print(train_x.shape)
print(validation_x.shape)
print(testing_x.shape)

(1800, 64, 64, 3)
(495, 64, 64, 3)
(1531, 64, 64, 3)


In [227]:
# Train the model, iterating on the data in batches of 32 samples
classifier.fit(train_x, train_y, epochs=110, batch_size=batch_size)

Epoch 1/110
1800/1800 [==============================] - 2s 1ms/step - loss: 0.8274 - acc: 0.5522
Epoch 2/110
1800/1800 [==============================] - 1s 582us/step - loss: 0.6622 - acc: 0.5978
Epoch 3/110
1800/1800 [==============================] - 1s 588us/step - loss: 0.6351 - acc: 0.6294
Epoch 4/110
1800/1800 [==============================] - 1s 585us/step - loss: 0.6243 - acc: 0.6256
Epoch 5/110
1800/1800 [==============================] - 1s 586us/step - loss: 0.6163 - acc: 0.6328
Epoch 6/110
1800/1800 [==============================] - 1s 583us/step - loss: 0.5987 - acc: 0.6461
Epoch 7/110
1800/1800 [==============================] - 1s 585us/step - loss: 0.5868 - acc: 0.6428
Epoch 8/110
1800/1800 [==============================] - 1s 585us/step - loss: 0.5672 - acc: 0.6589
Epoch 9/110
1800/1800 [==============================] - 1s 583us/step - loss: 0.5487 - acc: 0.6844
Epoch 10/110
1800/1800 [==============================] - 1s 582us/step - loss: 0.5226 - acc: 0.7150
E

1800/1800 [==============================] - 1s 586us/step - loss: 0.1783 - acc: 0.9294
Epoch 83/110
1800/1800 [==============================] - 1s 581us/step - loss: 0.1798 - acc: 0.9250
Epoch 84/110
1800/1800 [==============================] - 1s 584us/step - loss: 0.1585 - acc: 0.9350
Epoch 85/110
1800/1800 [==============================] - 1s 582us/step - loss: 0.1665 - acc: 0.9283
Epoch 86/110
1800/1800 [==============================] - 1s 580us/step - loss: 0.1730 - acc: 0.9283
Epoch 87/110
1800/1800 [==============================] - 1s 581us/step - loss: 0.1746 - acc: 0.9322
Epoch 88/110
1800/1800 [==============================] - 1s 583us/step - loss: 0.1687 - acc: 0.9306
Epoch 89/110
1800/1800 [==============================] - 1s 587us/step - loss: 0.1598 - acc: 0.9317
Epoch 90/110
1800/1800 [==============================] - 1s 586us/step - loss: 0.1499 - acc: 0.9378
Epoch 91/110
1800/1800 [==============================] - 1s 582us/step - loss: 0.1719 - acc: 0.9300
Epo

In [228]:
score = classifier.evaluate(validation_x, validation_y, batch_size=batch_size)

495/495 [==============================] - 0s 589us/step


In [229]:
score

[0.2608531713485718, 0.9010100960731506]

In [230]:
prediction_prob = list(classifier.predict(testing_x).transpose())[0]
prediction_classes = classifier.predict_classes(testing_x)

In [231]:
print(type(prediction_prob))
print(prediction_prob)
print(type(prediction_classes))
print(prediction_classes)

<class 'numpy.ndarray'>
[0.9997687  0.4150556  0.999995   ... 0.9329339  0.01756093 0.04174454]
<class 'numpy.ndarray'>
[[1]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]]


In [ ]:
print("name,invasive")
for i,j in zip(testing_name, prediction_prob):
    print(str(i)+","+str(j))